# Pytorch implementation of Google Landmark Recognition Challenge

## Needed imports

In [ ]:
use_gpu=True #change as required

In [ ]:
from torch import np as tnp# Torch wrapper for Numpy
import numpy as np
import os
from PIL import Image
import pickle
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.preprocessing import MultiLabelBinarizer
size=32,32
from PIL import ImageFile#needed for working with some images
ImageFile.LOAD_TRUNCATED_IMAGES = True

## Load data from files

In [ ]:
train_wd = os.path.join(os.getcwd(),'train')
test_wd = os.path.join(os.getcwd(), 'test')
print (train_wd)
print (test_wd)

In [ ]:
from os import listdir
from os.path import isfile, join

trainfiles = [f for f in listdir(train_wd) if isfile(join(train_wd, f))]
testfiles = [f for f in listdir(test_wd) if isfile(join(test_wd,f))]

In [ ]:
print(len(trainfiles)) # 235813
print(len(testfiles)) # 7166

In [ ]:
import csv
all_train = {}
iter = 0

with open(os.path.join(os.getcwd(), 'train.csv')) as train_csvfile:
    reader = csv.DictReader(train_csvfile)
    # create lookup dictionary for images that have been successfully downloaded
    for row in reader:
        all_train[row['id']] = row['landmark_id']

In [ ]:
print(len(all_train))

In [ ]:
#Only run this code if need to delete bad images. Otherwise takes time for no purpose. Rerun notebook after running this
#This deletes images that cause CNN to fail training. Run script until no errors (should only be needed once). Ensure this has no errors 
#Make sure to reload data from above
"""
import cv2
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
itr = 0
img_size = (32,32)
for filename in trainfiles:
    p=os.path.join(train_wd,filename)
    img = cv2.imread(p,0) # 0 second input for greyscale.

    # flatten image
    try:
        img = cv2.resize(img, img_size).flatten()
    except Exception:
        print (filename +" is a bad image. Removing from dataset. From opencv")
        os.remove(p)
        continue
    try:
        img=Image.open(p)
        img = img.convert('RGB')
        img = img.resize((32, 32), Image.ANTIALIAS)
    except Exception:
        print (filename +" is a bad image. Removing from dataset. From PIL")
        os.remove(p)
        continue
    itr = itr + 1
    if itr % 10000 == 0:
        print(itr)
"""

In [ ]:
#get list of labels to be able to split
train_labels=[]
for filename in trainfiles:
    fn = filename.replace('.jpg','')
    train_labels.append(all_train[fn])

## Cross Validation setup and Preprocessing

In [ ]:
#do not run for validation
X=np.array(trainfiles)
y=np.array(train_labels)
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest=train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
print (len (Xtest))

In [ ]:
#one hot encoding of labels
mlb = MultiLabelBinarizer()
mlb_temp=mlb.fit_transform(tnp.array(train_labels)).astype(tnp.float32)

In [ ]:
#https://www.kaggle.com/mratsim/starting-kit-for-pytorch-deep-learning

#Class loads and transforms image files and labels as needed
#This class allows not all images to be stored at once in memory. Handles batches well. 
#Do all image changes here (though Augmentation pipeline is preferred to be used)

class LoadDataset(Dataset):

    def __init__(self, path, data,  transform=None):
    
        #tmp_df = pd.read_csv(csv_path)
        
        
        #self.mlb = MultiLabelBinarizer()
        self.img_path = path
        #self.img_ext = ".jpg"
        self.transform = transform

        self.X_train = data
        #self.y_train = self.mlb.fit_transform(all_train['id'].str.split()).astype(np.float32)
        self.train_labels = []
        itr = 0
        img_size = (32,32)
        for filename in self.X_train:

            # find targets
            fn = filename.replace('.jpg','')
            train_labels.append(all_train[fn])
            
            itr = itr + 1
            #if itr % 10000 == 0:
            #    print(itr)
        #self.y_train=self.mlb.fit_transform(tnp.array(train_labels)).astype(tnp.float32)#one hot encoding reverse by 
        #mlb.inverse_transform(predictions
        self.y_train=mlb_temp
        #self.y_train=np.array(list(all_labels), dtype=np.int32)
        
    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_path , self.X_train[index]))
        img = img.convert('RGB')
        img = img.resize((32, 32), Image.ANTIALIAS)
        if self.transform is not None:
            img = self.transform(img)
        #label = torch.from_numpy(self.y_train[index])
        label=torch.from_numpy(np.array([self.y_train[index]], dtype=np.int32)).float()
        return img, label

    def __len__(self):
        return len(self.X_train)

## Set up pytorch train Dataloader

In [ ]:
#set up transformations for images and set u data loader

transform = transforms.Compose(
     [transforms.Resize(32),
     #transforms.RandomCrop(256),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dset_train = LoadDataset(train_wd, Xtrain, transform)

In [ ]:
if not use_gpu:
    train_loader = DataLoader(dset_train,
                          batch_size=50,#can be changed depending if GPU has enough memory
                          shuffle=True,
                          num_workers=0 # 0 works higher seems not to prrobably jupyter notebook
                          #pin_memory=True # CUDA only
                         )
else:
    train_loader = DataLoader(dset_train,
                          batch_size=50,#can be changed depending if GPU has enough memory
                          shuffle=True,
                          num_workers=0, # 1 for CUDA (0 always works) must be 0 
                          pin_memory=True # CUDA only
                         )

## Set up CNN

In [ ]:
#CNN class that assumes size of images will be 256x256 and labels are of tensor size 10. 
#Must change values here if size changes.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        #self.fc1 = nn.Linear(2304, 256)
        #self.fc1=nn.Linear(246016,256)
        self.fc1=nn.Linear(2304,50)
        #self.fc2 = nn.Linear(256, 10)
        self.fc2 = nn.Linear(50, 10)
        #self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.size(0), -1) # Flatten layer
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.sigmoid(x)

if not use_gpu:
    model = Net() # On CPU
else:
    model = Net().cuda() # On GPU

In [ ]:
#run this for both training and validation
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion =nn.CrossEntropyLoss()

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda(async=True) # On GPU
        data, target = Variable (data), Variable (target)
        optimizer.zero_grad()
        output = model(data)
        #loss = F.CrossEntropyLoss(output, target)
        loss = criterion(output, torch.max(target, 1)[1])
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
        del data, target
        if use_gpu:
            torch.cuda.empty_cache()

## Trains data

In [ ]:
for epoch in range(0, 1):
    train(epoch)
#model.eval() for test

In [ ]:
#run to clear up memory if want to go straight to validation withouyt saving
del train_loader
if use_gpu:
    torch.cuda.empty_cache()

## Save model 

In [ ]:
#run so can keep train and test data seperate if need restart kernel (out of memory)
torch.save(model.state_dict(), 'checkpoint.pth')
with open("test_filenames.txt", "wb") as fp:   #Pickling
    pickle.dump(Xtest, fp)

## Load model

In [ ]:
#https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch

#only run if need load in data and model from disk (typically from a kernel restart)
#Or just run it because you can
# Warning: Running this without having both a checkpoint in current working directory and the test_filenames 
if not use_gpu:
    model=Net()
else:
    model=Net().cuda()
model.load_state_dict(torch.load('checkpoint.pth'))
with open("test_filenames.txt", "rb") as fp:   # Unpickling
    Xtest = pickle.load(fp)

## Validation

In [ ]:
losses = []
model.eval()
#torch.cuda.empty_cache()
transform = transforms.Compose(
     [transforms.Resize(32),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dset_test = LoadDataset(train_wd, Xtest, transform)
if use_gpu:
    validation_loader = DataLoader(dset_test,
                          batch_size=50,
                          shuffle=False,
                          num_workers=0, # 1 for CUDA
                          pin_memory=True # CUDA only
                         )
else:
    validation_loader = DataLoader(dset_test,
                          batch_size=50,
                          shuffle=False,
                          num_workers=0 # 1 for CUDA
                          #pin_memory=True # CUDA only
                         )

In [ ]:
batch_size=50
from sklearn.metrics import f1_score
def validate():
    count=0
    accuracy=0
    for batch_idx, (data, target) in enumerate(validation_loader):
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda(async=True) # On GPU
        data, target = Variable(data), Variable(target)
        output = model(data)
        #print(target)
        #print(mlb.inverse_transform(output.data))
        #print(output.data.numpy())#take 1 row at a time transform it and can compare labels or just compare to target
        #accuracy = 100 * correct / total
        #print(accuracy)
        loss = criterion(output, torch.max(target, 1)[1])
        losses.append(loss.data[0])
        if batch_idx%100 == 0:
            pass
        t=None
        o=None
        if use_gpu:
            t=target.cpu().data.numpy()
            o=output.cpu().data.numpy()
        else:
            t=target.data.numpy()
            o=output.data.numpy()
        #print (t[0])
        #print (o[0])
        del data,target
        for i in range(batch_size):#todo test this out
            count +=1
            try:
                if np.array_equal(t[i],o[i]):
                    accuracy += 1
            except Exception:
                continue
        print ("Batch number ", batch_idx, " just passed")
        #acc=f1_score(target, output, average='macro')  
        #print("Batch: ", batch_idx,"Accuracy: ", acc)
    print(sum(losses)/len(losses))
    print ("accuracy: ", accuracy/count)

In [ ]:
validate()
#look at visdom for visualization

In [ ]:
del validation_loader
if use_gpu:
    torch.cuda.empty_cache()

## Examine GPU memory

In [ ]:
#only run if want to see GPU memory allocation
#http://forums.fast.ai/t/gpu-memory-not-being-freed-after-training-is-over/10265/8
def pretty_size(size):
    """Pretty prints a torch.Size object"""
    assert(isinstance(size, torch.Size))
    return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
    """Prints a list of the Tensors being tracked by the garbage collector."""
    import gc
    total_size = 0
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj):
                if not gpu_only or obj.is_cuda:
                    print("%s:%s%s %s" % (type(obj).__name__, 
                                          " GPU" if obj.is_cuda else "",
                                        " pinned" if obj.is_pinned else "",
                                          pretty_size(obj.size())))
                    total_size += obj.numel()
            elif hasattr(obj, "data") and torch.is_tensor(obj.data):
                if not gpu_only or obj.is_cuda:
                    print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
                                                    type(obj.data).__name__, 
                                                    " GPU" if obj.is_cuda else "",
                                                    " pinned" if obj.data.is_pinned else "",
                                                    " grad" if obj.requires_grad else "", 
                                                    " volatile" if obj.volatile else "",
                                                    pretty_size(obj.data.size())))
                    total_size += obj.data.numel()
        except Exception as e:
            pass        
    print("Total size:", total_size)
dump_tensors(True)